In [7]:
import simulaqron
from simulaqron.network import Network
from cqc.pythonLib import CQCConnection
from cqc.pythonLib import qubit
import threading

In [8]:
def alice():
    with CQCConnection("Alice", network_name="teleportation") as alice:
        # setup entanglement
        qA = alice.createEPR("Bob")

        # qubit to be teleported
        qT = qubit(alice)
        qT.H()

        # bell measurement
        qT.cnot(qA)
        qT.H()

        # measure for correction
        a = qT.measure()
        b = qA.measure()
        alice.sendClassical("Bob", [a, b])

def bob():
    with CQCConnection("Bob", network_name="teleportation") as bob:
        # setup entanglement
        qB = bob.recvEPR()

        # receive corrections
        msg = bob.recvClassical()
        data = list(msg)
        a = data[0]
        b = data[1]

        # apply corrections
        if b == 1: qB.X()
        if a == 1: qB.Z()

        # measure
        m = qB.measure()
        print(f"Bob's meas. out.: {m}")

In [9]:
nodes = ["Alice", "Bob"]
topology = {"Alice": ["Bob"], "Bob": ["Alice"]}
network_name = "teleportation"
network = Network(name=network_name, nodes=nodes, topology=topology, force=True) 
network.start()


for i in range(10):
    aliceThread = threading.Thread(target=alice)
    bobThread = threading.Thread(target=bob)

    aliceThread.start()
    bobThread.start()

    aliceThread.join()
    bobThread.join()

Bob's meas. out.: 1
Bob's meas. out.: 1
Bob's meas. out.: 1
Bob's meas. out.: 1
Bob's meas. out.: 0
Bob's meas. out.: 0
Bob's meas. out.: 0
Bob's meas. out.: 0
Bob's meas. out.: 0
Bob's meas. out.: 0
